In [1]:
import tensorflow.keras as keras
import tensorflow.keras.layers as KL
import tensorflow as tf
from tensorflow.keras.layers import concatenate, Conv2D, Dense, Flatten, Layer, MaxPooling2D
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import tensorflow.keras as keras

%load_ext autoreload
%autoreload 2

%matplotlib notebook

%load_ext tensorboard
%tensorboard --logdir logs --bind_all

sys.path.append('..')

In [2]:
mypath = "./noise_generation/tmp"
from os import listdir
from os.path import isfile, join
import cv2
import numpy as np

num_of_pictures = 50

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
original_images = np.empty([num_of_pictures,256,256,3])
noised_images = np.empty([num_of_pictures,256,256,3])

iteration = 0
countOdd = 0
countEven = 0

for q in onlyfiles[:num_of_pictures]:
    name = mypath+"/"+q
   # images.append(cv2.imread(name))
   
    if (iteration % 2 == 0):
        noised_images[countOdd] = cv2.imread(name)
        noised_images[countOdd] /= 255.0
        countOdd = countOdd + 1    
    else:
        original_images[countEven] = cv2.imread(name)
        original_images[countEven] /= 255.0
        countEven = countEven + 1
    iteration += 1
    
split = int(original_images.size*0.7)

train_orig = original_images[:split]
train_noised = noised_images[:split]
                        
test_orig = original_images[split:]
test_noised = noised_images[split:]
                        
    


In [6]:
# NOT WORKING


import tensorflow.keras as keras
from tensorflow.keras.layers import concatenate, Conv2D, Dense, Flatten, Layer, MaxPooling2D

class Inception(Layer):
    def __init__(self, filters, activation):
        """
        :param filters: How many filters are in the convolutional layers within this layer.
        :param activation: What is the activation function used
        """
        super(Inception, self).__init__()

        # Layer 1
        self.conv1 = Conv2D(
            filters=filters,
            kernel_size=1,
            padding='same',
            activation=activation)
        
        self.conv2 = Conv2D(
            filters=filters,
            kernel_size=1,
            padding='same',
            activation=activation)
        
        self.maxpool = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding ="same")
        
        #Layer 2
        
        self.conv3 = Conv2D(
            filters=filters,
            kernel_size=3,
            padding='same',
            activation=activation)
        
        self.conv4 = Conv2D(
            filters=filters,
            kernel_size=5,
            padding='same',
            activation=activation)   
        
        self.conv5 = Conv2D(
            filters=filters,
            kernel_size=1,
            padding='same',
            activation=activation)        
        
        self.conv6 = Conv2D(
            filters=filters,
            kernel_size=1,
            padding='same',
            activation=activation)   

    def call(self, x):
        # layer 1
        l1_1 = self.conv1(x)
        l1_2 = self.conv2(x)
        l1_m = self.maxpool(x)
        lm = self.conv6(x)
        
        # layer 2
        l2_1 = self.conv3(l1_1)
        l2_2 = self.conv4(l1_2)
        l2_3 = self.conv5(l1_m)
        
        # final
        return concatenate([lm, l2_1, l2_2, l2_3])


class InceptionNet(keras.Model):
    """
    Inception version of the simple CNN we used previously. You do not need to change anything here.
    """

    def __init__(self, filters, dim_output):
        super(InceptionNet, self).__init__()
        self.model_layers = [
            Inception(
                filters=filters,
                activation='relu'),
            MaxPooling2D(pool_size=(2, 2)),
            Inception(
                filters=filters,
                activation='relu'),
            MaxPooling2D(pool_size=(2, 2)),
            Inception(
                filters=filters,
                activation='relu'),
            MaxPooling2D(pool_size=(2, 2)),
            Flatten(),
            Dense(
                units=256*256,
                activation='relu'),
            Dense(
                units=dim_output,
                activation='softmax')
        ]

    def call(self, x):
        for layer in self.model_layers:
            x = layer(x)
        return x


In [ ]:
# NOT WORKING


model = InceptionNet(
    filters=16,
    dim_output=256)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.003),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

"""
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir=os.path.join("logs", "denoising"),
        histogram_freq=1,
        profile_batch=0)
]
"""

model.fit(
    x=train_noised,
    y=train_orig,
    batch_size=16,
    validation_data=(test_noised, test_orig),
   # callbacks=callbacks,
    epochs=30)  